## Preparación del entorno.

Si no estamos parados en el repo, clonar y cd al repo. Esto nos permite usar el mismo notebook tanto local como en Google Colab.

In [ ]:
import os

REPO_NAME = "lab7"
if REPO_NAME not in os.getcwd():
  if not os.path.exists(REPO_NAME):
    !git clone https://github.com/FCEIA-AAII/{REPO_NAME}.git
  os.chdir(REPO_NAME)


Importar librerías

In [ ]:
import numpy as np
from pathlib import Path
import tensorflow as tf
from keras.layers import Input, Dense, GlobalMaxPooling2D
from tensorflow.python.keras.callbacks import ModelCheckpoint, ReduceLROnPlateau, EarlyStopping 
import matplotlib.pyplot as plt
from PIL import Image
import cv2

Establecer GPU por defecto en caso de estar disponible.

In [ ]:
# Configurar para que TensorFlow utilice la GPU por defecto
gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
    try:
        # Configurar para que TensorFlow asigne memoria dinámicamente
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)
        # Especificar la GPU por defecto
        logical_gpus = tf.config.experimental.list_logical_devices('GPU')
        print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPUs")
    except RuntimeError as e:
        # Manejar error
        print(e)

Cargar dataset:

In [ ]:
# Directorio de los datos
TRAIN_DATA_DIRECTORY = Path("dataset/train")
VALIDATION_DATA_DIRECTORY = Path("dataset/validation")

# Tamaño del lote (batch size)
BATCH_SIZE = 32
# Tamaño de las imágenes
IMAGE_HEIGHT = 224
IMAGE_WIDTH = 224

# Carga los datos de entrenamiento y validación
train_ds = tf.keras.utils.image_dataset_from_directory(
    TRAIN_DATA_DIRECTORY,
    label_mode="categorical",
    image_size=(IMAGE_HEIGHT, IMAGE_WIDTH),
    batch_size=BATCH_SIZE)

val_ds = tf.keras.utils.image_dataset_from_directory(
    VALIDATION_DATA_DIRECTORY,
    label_mode="categorical",
    image_size=(IMAGE_HEIGHT, IMAGE_WIDTH),
    batch_size=BATCH_SIZE)

Inspeccionar las clases:

In [ ]:
# Obtiene los nombres de las clases
class_names = train_ds.class_names
num_classes = len(class_names)
print(class_names)

Reducimos el tamaño del dataset para emular un escenario real donde no tenemos muchos datos.

In [ ]:
train_ds = train_ds.take(200)
val_ds = val_ds.take(100)

Visualizar los datos:

In [ ]:
# Muestra algunas imágenes de ejemplo
plt.figure(figsize=(10, 10))
for images, labels in train_ds.take(1):
  for i in range(9):
    ax = plt.subplot(3, 3, i + 1)
    plt.imshow(images[i].numpy().astype("uint8"))
    class_idx = list(labels[i]).index(1)
    plt.title(class_names[class_idx])
    plt.axis("off")

Definir la arquitectura de la red:

In [ ]:
def build_model(input_shape, num_classes):
    base_model = tf.keras.applications.EfficientNetB0(input_shape=input_shape,
                                                include_top=False,
                                                weights='imagenet')
    base_model.trainable = False

    i = Input(shape=input_shape)
    x = base_model(i, training=False)
    x = GlobalMaxPooling2D()(x)
    x = Dense(num_classes, activation='softmax')(x)

    return tf.keras.Model(i, x)

Construir el modelo:

In [ ]:
print("Building model")
model = build_model((IMAGE_HEIGHT, IMAGE_WIDTH, 3), num_classes)

model.compile(
    optimizer='adam',
    loss="categorical_crossentropy",
    metrics=['accuracy'])

Resumen del modelo:

In [ ]:
model.summary()

Entrenar el modelo:

In [ ]:
early_stopping = EarlyStopping(monitor="val_loss", patience=10, verbose=0, mode="min")
checkpoint_acc = ModelCheckpoint(
    "model-e{epoch:02d}-loss{val_loss:.3f}-acc{val_accuracy:.3f}",
    save_best_only=True,
    monitor="val_accuracy",
    initial_value_threshold=0.7,
    mode="max",
)
reduce_lr = ReduceLROnPlateau(
    monitor="loss", factor=0.5, patience=20, verbose=1, epsilon=1e-4, mode="min"
)

# Número de épocas de entrenamiento
EPOCHS = 50
# Entrena el modelo
history = model.fit(
    train_ds,
    validation_data=val_ds,
    epochs=EPOCHS,
    batch_size=BATCH_SIZE,
    callbacks=[checkpoint_acc, reduce_lr, early_stopping],
)


Visualizar resultados de entrenamiento:

In [ ]:
# Grafica la precisión y pérdida de entrenamiento y validación
acc = history.history['accuracy']
val_acc = history.history['val_accuracy']

loss = history.history['loss']
val_loss = history.history['val_loss']

epochs_range = range(EPOCHS)

plt.figure(figsize=(8, 8))
plt.subplot(1, 2, 1)
plt.plot(epochs_range, acc, label='Training Accuracy')
plt.plot(epochs_range, val_acc, label='Validation Accuracy')
plt.legend(loc='lower right')
plt.title('Training and Validation Accuracy')

plt.subplot(1, 2, 2)
plt.plot(epochs_range, loss, label='Training Loss')
plt.plot(epochs_range, val_loss, label='Validation Loss')
plt.legend(loc='upper right')
plt.title('Training and Validation Loss')
plt.show()

Algoritmo de clasificacion:

In [ ]:
def classify_img(img):
    img_pred = cv2.cvtColor(img.copy(), cv2.COLOR_BGR2RGB)  # Convert head crop to RGB format
    img_pred = cv2.resize(img_pred, (IMAGE_WIDTH, IMAGE_WIDTH))  # Resize the image
    img_pred = np.expand_dims(img_pred, axis=0)  # Expand dimensions to create batch
    predictions = model(img_pred)  # Perform hat classification
    predicted_score = np.max(predictions) # Get the max score
    predicted_class_index = np.argmax(predictions)  # Get index of predicted class
    predicted_class = class_names[predicted_class_index]  # Get predicted class
    return predicted_class, predicted_score # Return predicted hat class

Algoritmo de ventana deslizante con tamaño fijo:

In [ ]:
def sliding_window(image, step_size, window_size):
    """
    Genera regiones de una imagen utilizando el algoritmo de ventana deslizante.

    Args:
        image: Imagen de entrada.
        step_size: Tamaño del paso para desplazar la ventana.
        window_size: Tamaño de la ventana (altura, ancho).

    Returns:
        Una lista de tuplas (x, y, ventana) donde (x, y) es la esquina superior izquierda
        de la ventana y ventana es la región de la imagen cubierta por la ventana.
    """
    windows = list()
    # Itera sobre las coordenadas (x, y) de la imagen con el paso especificado
    for y in range(0, image.shape[0] - window_size[0] + 1, step_size):
        for x in range(0, image.shape[1] - window_size[1] + 1, step_size):
            # Define la región de la imagen cubierta por la ventana
            crop = image[y:y+window_size[0], x:x+window_size[1]]
            # Aplica el modelo de clasificacion
            predicted_class, predicted_score = classify_img(crop)
            # Crea la tupla window = (x, y, w, h, class, score)
            window = (x, y, window_size[1], window_size[0], predicted_class, predicted_score)
            windows.append(window)
    return windows

Algoritmo de ventana deslizante con tamaño variable:

In [ ]:
def sliding_window_variable(image):
    """
    Genera todas las regiones posibles de una imagen utilizando iteradamente el algoritmo de ventana deslizante.

    Args:
        image: Imagen de entrada.

    Returns:
        Una lista de tuplas (x, y, w, h, class, score) donde (x, y) es la esquina superior izquierda
        de la ventana, w y h son el ancho y alto de la ventana, respectivamente, y class y score son
        la clase predicha y la puntuación correspondiente para esa ventana.
    """

Algoritmo NMS:

In [ ]:
def nms(windows, threshold):
    """
    Aplica el algoritmo de supresión de no máximos (NMS) a las ventanas.

    Args:
        windows: Lista de tuplas que representan las ventanas, cada tupla en el formato (x, y, w, h, class, score).
        threshold: Umbral de solapamiento para decidir si dos ventanas deben suprimirse.

    Returns:
        Una lista de ventanas después de aplicar NMS.
    """

Algoritmo para dibujar detecciones:

In [ ]:
def draw_bboxes(filtered_windows, image):
    """
    Dibuja los bounding boxes de las ventanas filtradas con su clase sobre la imagen.

    Args:
        filtered_windows: Lista de tuplas que representan las ventanas, cada tupla en el formato (x, y, w, h, class, score).

    Returns:
        Una imagen con los bouding boxes y sus respectivas clases.
    """

Implementacion final:
 - Comparar resultados de sliding window con tamaño fijo para distintos valores de step_size y window_size.
 - Comparar resultados de sliding window con tamaño fijo y variable.
 - Aplicar NMS sobre los resultados del sliding window con tamaño variable.

En todos los casos aplicar un treshold de score sobre las windows y dibujar las detecciones sobre las imagenes.
Utilizar como imagenes de entrada las que se encuentran en la carpeta ./detection-test-images/